In [1]:
%%capture
%load_ext autoreload
%autoreload 2
import itertools as itt
import os
import sys
import holoviews as hv
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.core.display import display
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
# Importing the custom helper functions
from calcium_behavior_alignment import *

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

## Step 0 Initialize Variables

In [3]:
exp_path = "/scratch/09117/xz6783/minian/Satiation/"
id_path = "/scratch/09117/xz6783/minian/Satiation/Session Combined/"
beh_path = "/scratch/09117/xz6783/minian/Behavior_Files/"
experiment = "Satiation Protocol"

# exp_path = "/scratch/09656/dhh3/Satiation/"
# id_path = "/scratch/09656/dhh3/calcium-behavior-alignment/"
# beh_path = "/scratch/09656/dhh3/Behavior_Files/"

minian_path = "."

files = os.listdir(id_path)

animal_ids = [file for file in files if os.path.isdir(os.path.join(id_path, file)) and len(file) == 3 and file[0].isdigit()]

print(len(animal_ids))
animal_ids

19


['4B6',
 '2I5',
 '7B6',
 '6H6',
 '4N6',
 '8H6',
 '9M6',
 '2S5',
 '2N6',
 '6S5',
 '7M6',
 '1B6',
 '6M6',
 '1H6',
 '5B6',
 '1S5',
 '1I5',
 '7H6',
 '2Z4']

In [4]:
# animal_id ="7B6"   #change name for each animal
# minian_path = "."
# dpath = "/scratch/09656/dhh3/Satiation/Session Combined/1B6/"
# minian_ds_path = os.path.join(dpath, "minian")
# intpath = "./minian_intermediate"

# #change for each experiment:
# output_path_spike = r"E:\Xu\Miniscope\PL\Raw Data\Satiation\Session Combined\Spikes"   
# output_path_calcium = r"E:\Xu\Miniscope\PL\Raw Data\Satiation\Session Combined\Calcium"   
# output_path_calcium_hungry = r"E:\Xu\Miniscope\PL\Raw Data\Satiation\Session Combined\Calcium\Hungry"
# output_path_calcium_satiated = r"E:\Xu\Miniscope\PL\Raw Data\Satiation\Session Combined\Calcium\Satiated"

# subset1 = dict(frame=slice(0,23237))
# subset2 = dict(frame=slice(23238, None))



# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MINIAN_INTERMEDIATE"] = intpath

In [5]:
# # Set up Initial Basic Parameters#
# minian_path = "."
# dpath = "."
# minian_ds_path = os.path.join(dpath, "minian")
# intpath = "./minian_intermediate"
# subset = dict(frame=slice(0, None))
# subset_mc = None
# interactive = True
# output_size = 100
# n_workers = int(os.getenv("MINIAN_NWORKERS", 30))
# param_save_minian = {
#     "dpath": minian_ds_path,
#     "meta_dict": dict(session=-1, animal=-2),
#     "overwrite": True,
# }

# # Pre-processing Parameters#
# param_load_videos = {
#     "pattern": "[0-9]+\.avi$",
#     "dtype": np.uint8,
#     "downsample": dict(frame=1, height=1, width=1),
#     "downsample_strategy": "subset",
# }
# param_denoise = {"method": "median", "ksize": 7}
# param_background_removal = {"method": "tophat", "wnd": 15}

# # Motion Correction Parameters#
# subset_mc = None
# param_estimate_motion = {"dim": "frame"}

# # Initialization Parameters#
# param_seeds_init = {
#     "wnd_size": 1000,
#     "method": "rolling",
#     "stp_size": 500,
#     "max_wnd": 15,
#     "diff_thres": 3,
# }
# param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
# param_ks_refine = {"sig": 0.05}
# param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
# param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
# param_init_merge = {"thres_corr": 0.8}

# # CNMF Parameters#
# param_get_noise = {"noise_range": (0.06, 0.5)}
# param_first_spatial = {
#     "dl_wnd": 10,
#     "sparse_penal": 0.01,
#     "size_thres": (25, None),
# }
# param_first_temporal = {
#     "noise_freq": 0.06,
#     "sparse_penal": 1,
#     "p": 1,
#     "add_lag": 20,
#     "jac_thres": 0.2,
# }
# param_first_merge = {"thres_corr": 0.8}
# param_second_spatial = {
#     "dl_wnd": 10,
#     "sparse_penal": 0.01,
#     "size_thres": (25, None),
# }
# param_second_temporal = {
#     "noise_freq": 0.06,
#     "sparse_penal": 1,
#     "p": 1,
#     "add_lag": 20,
#     "jac_thres": 0.4,
# }

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MINIAN_INTERMEDIATE"] = intpath

## Step 1: Parse Scope Times

In [6]:
# Parsing
scope_times = parse_scope_times(exp_path=exp_path,
                               id_path=id_path)

scope_times

{'4B6': {'4B6_2022_08_31_13_39_26':        Frame Number  Time Stamp (s)  Buffer Index
  0                 0          -0.023             0
  1                 1           0.049             0
  2                 2           0.079             0
  3                 3           0.128             0
  4                 4           0.179             0
  ...             ...             ...           ...
  14816         14816         749.773             0
  14817         14817         749.824             0
  14818         14818         749.874             0
  14819         14819         749.926             0
  14820         14820         749.976             0
  
  [14821 rows x 3 columns],
  '4B6_2022_08_31_15_02_01':        Frame Number  Time Stamp (s)  Buffer Index
  0                 0          -0.009             0
  1                 1           0.043             0
  2                 2           0.093             0
  3                 3           0.143             0
  4                 4   

## Step 2: Process animal data based on ID

In [9]:
behavior_dict = parse_behavior_times(beh_path=beh_path, experiment=experiment, animal_ids=animal_ids)

In [10]:
behavior_dict['2Z4']

,Time (s),Speed (m/s),Freezing,In Food Area,Bar Press active,Miniscope record active,Tone active
0,0.000,NaN,0,0,0,0,0
1,1.311,0.010,0,0,0,0,0
2,1.454,0.007,0,0,0,0,0
3,1.583,0.005,0,0,0,0,0
4,1.711,0.004,0,0,0,0,0
...,...,...,...,...,...,...,...
63241,4559.860,0.040,0,1,0,1,0
63242,4559.891,0.040,0,1,0,0,0
63243,4559.939,0.030,0,1,0,0,0
63244,4559.963,0.030,0,1,0,1,0


In [11]:
scope_times['2Z4']

{'2Z4_2021_09_20_10_14_35':        Frame Number  Time Stamp (s)  Buffer Index
 0                 0          -0.032             0
 1                 1           0.026             0
 2                 2           0.069             0
 3                 3           0.123             0
 4                 4           0.169             0
 ...             ...             ...           ...
 14815         14815         749.723             0
 14816         14816         749.767             0
 14817         14817         749.819             0
 14818         14818         749.869             0
 14819         14819         749.920             0
 
 [14820 rows x 3 columns],
 '2Z4_2021_09_20_11_22_48':        Frame Number  Time Stamp (s)  Buffer Index
 0                 0          -0.024             0
 1                 1           0.034             0
 2                 2           0.075             0
 3                 3           0.125             0
 4                 4           0.182             0

In [12]:
list(scope_times['2Z4'].keys())[1]

'2Z4_2021_09_20_11_22_48'

## Step 3: Process spikes and calcium

In [7]:
dpath = os.path.join(id_path, "5B6")
minian_ds_path = os.path.join(dpath, "minian")
minian_ds = open_minian(minian_ds_path)

In [8]:
minian_ds_path

'/scratch/09117/xz6783/minian/Satiation/Session Combined/5B6/minian'

In [9]:
tracenew_spike, labelsnew_spike, tracenew_calcium, labelsnew_calcium = process_spikes_and_calcium(minian_ds, verbose=True)

Deleted 39 neurons out of the original 68 neurons, 29 neurons remain
Deleted 39 neurons out of the original 68 neurons, 29 neurons remain


## Step 4: Align and Interpolate

In [13]:
timestamps, behavior = combine_datasets(scope_times, behavior_dict, "2Z4", verbose=True)

Combining Data (2Z4): Behavior end of recording: 4528.854
Combining Data (2Z4): Timestamp last value 4529.9529999999995
Combining Data (2Z4): breaks: [20555] -> recording sections: [(215, 20555), (43088, 63245)]


In [14]:
timestamps

,Frame Number,Time Stamp (s),Buffer Index
0,0,-0.032,0
1,1,0.026,0
2,2,0.069,0
3,3,0.123,0
4,4,0.169,0
...,...,...,...
29931,15111,4529.752,0
29932,15112,4529.801,0
29933,15113,4529.852,0
29934,15114,4529.903,0


In [15]:
behavior

,Time (s),Speed (m/s),Freezing,In Food Area,Bar Press active,Miniscope record active,Tone active
0,-30.137,NaN,0,0,0,0,0
1,-28.826,0.010,0,0,0,0,0
2,-28.683,0.007,0,0,0,0,0
3,-28.554,0.005,0,0,0,0,0
4,-28.426,0.004,0,0,0,0,0
...,...,...,...,...,...,...,...
63241,4529.723,0.040,0,1,0,1,0
63242,4529.754,0.040,0,1,0,0,0
63243,4529.802,0.030,0,1,0,0,0
63244,4529.826,0.030,0,1,0,1,0


### Spikes

In [20]:
tracealigned, labelsaligned = align_and_interpolate(animal_timestamps=timestamps, 
                                                    animal_behavior=behavior, 
                                                    tracenew=tracenew_spike, 
                                                    labelsnew=labelsnew_spike)

In [21]:
output_path_spike = os.path.join(id_path, "Spikes")

# Save spike
save_trace_and_labels(tracealigned, labelsaligned, output_path_spike, "5B6")

In [22]:
pd.read_pickle(os.path.join(output_path_spike, "5B6.pkl"))

,0,1,2,3,4,5,6,7,8,9,...,29843,29844,29845,29846,29847,29848,29849,29850,29851,29852
2,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
3,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
4,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
6,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
10,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
11,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
29,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
16,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
17,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
18,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000


### Calcium

In [161]:
tracealigned, labelsaligned = align_and_interpolate(animal_timestamps=timestamps, 
                                                    animal_behavior=behavior, 
                                                    tracenew=tracenew_calcium, 
                                                    labelsnew=labelsnew_calcium)

tracenew shape(29927, 139)
Cue(29937,)
Bar(29937,)
CATIME(29937,)
ENTERED PADDING
tracenew_padded:(29937, 139)


In [163]:
output_path_calcium = os.path.join(id_path, "Calcium")

# Save calcium
save_trace_and_labels(tracealigned, labelsaligned, output_path_calcium, "2IH")

## Try Main Execute on All Animal IDs

In [7]:
execute(exp_path=exp_path, id_path=id_path, beh_path=beh_path, experiment=experiment, num_processes=1, verbose=True)

Experiment path: /scratch/09117/xz6783/minian/Satiation/
Id path: /scratch/09117/xz6783/minian/Satiation/Session Combined/
Behavior path: /scratch/09117/xz6783/minian/Behavior_Files/
Found 19 animal IDs
Animal IDs: dict_keys(['4B6', '2I5', '7B6', '6H6', '4N6', '8H6', '9M6', '2S5', '2N6', '6S5', '7M6', '1B6', '6M6', '1H6', '5B6', '1S5', '1I5', '7H6', '2Z4'])
Found 14 animal IDs with two entries
Found 5 animal IDs with one entry
Erronous animal IDs: None
Processing and aligning 19 animal IDs with 1 process(es)


Processing animal ID 4B6
Combining Data (4B6): Behavior end of recording: 4528.619
Combining Data (4B6): Timestamp last value 4518.2210000000005
Combining Data (4B6): breaks: [20670 30283 40075] -> recording sections: [(220, 20670), (41700, 62236)]
Saved 4B6 Calciums to /scratch/09117/xz6783/minian/Satiation/Session Combined/Calcium
Saved 4B6 Spikes to /scratch/09117/xz6783/minian/Satiation/Session Combined/Spikes
Successfully aligned animal ID 4B6


Processing animal ID 2I5
Comb

Saved 7H6 Spikes to /scratch/09117/xz6783/minian/Satiation/Session Combined/Spikes
Successfully aligned animal ID 7H6


Processing animal ID 2Z4
Combining Data (2Z4): Behavior end of recording: 4528.854
Combining Data (2Z4): Timestamp last value 4529.9529999999995
Combining Data (2Z4): breaks: [20555] -> recording sections: [(215, 20555), (43088, 63245)]
Saved 2Z4 Calciums to /scratch/09117/xz6783/minian/Satiation/Session Combined/Calcium
Saved 2Z4 Spikes to /scratch/09117/xz6783/minian/Satiation/Session Combined/Spikes
Successfully aligned animal ID 2Z4
Successfully processed and aligned 19 animal IDs: ['4B6', '2I5', '7B6', '6H6', '4N6', '8H6', '9M6', '2S5', '2N6', '6S5', '7M6', '1B6', '6M6', '1H6', '5B6', '1S5', '1I5', '7H6', '2Z4']
Failed to process and align 0 animal IDs: []


[(True, '4B6'),
 (True, '2I5'),
 (True, '7B6'),
 (True, '6H6'),
 (True, '4N6'),
 (True, '8H6'),
 (True, '9M6'),
 (True, '2S5'),
 (True, '2N6'),
 (True, '6S5'),
 (True, '7M6'),
 (True, '1B6'),
 (True, '6M6'),
 (True, '1H6'),
 (True, '5B6'),
 (True, '1S5'),
 (True, '1I5'),
 (True, '7H6'),
 (True, '2Z4')]